In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, countDistinct, sum, when, desc, date_format

# --- 설정 (Configuration) ---
MINIO_ENDPOINT = "http://titan-minio:9000"
ACCESS_KEY = "minioadmin"
SECRET_KEY = "minioadmin"

# 1. Spark 세션 생성
spark = SparkSession.builder \
    .appName("TitanLog-ETL-SilverToGold") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,com.amazonaws:aws-java-sdk-bundle:1.12.533") \
    .config("spark.hadoop.fs.s3a.endpoint", MINIO_ENDPOINT) \
    .config("spark.hadoop.fs.s3a.access.key", ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.secret.key", SECRET_KEY) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .getOrCreate()

# 2. Silver 데이터(Parquet) 읽기
# Parquet는 스키마가 저장되어 있어서 따로 타입을 지정할 필요가 없습니다. (개꿀!)
print("Silver 데이터 로딩 중...")
df = spark.read.parquet("s3a://silver/log_v1")
df.cache() # 데이터를 메모리에 올려서 반복 분석 속도 UP!

# ==========================================
# 분석 1: 일별 트래픽 (Daily Traffic)
# - PV (Page View): 전체 클릭 수
# - UV (Unique Visitor): 방문한 사람 수 (중복 제거)
# ==========================================
print("1. 일별 트래픽 분석 중...")

daily_traffic_df = df \
    .withColumn("date", date_format(col("event_time"), "yyyy-MM-dd")) \
    .groupBy("date") \
    .agg(
        count("event_id").alias("total_pv"),
        countDistinct("user_id").alias("daily_active_users (DAU)")
    ) \
    .orderBy("date")

daily_traffic_df.show()

# Gold 저장
daily_traffic_df.write.mode("overwrite").parquet("s3a://gold/daily_traffic")


# ==========================================
# 분석 2: 상품별 판매 랭킹 (Top Sellers)
# - event_type이 'purchase'인 것만 필터링해서 카운트
# ==========================================
print("2. 상품 판매 랭킹 분석 중...")

top_sellers_df = df \
    .filter(col("event_type") == "purchase") \
    .groupBy("item_id") \
    .count() \
    .withColumnRenamed("count", "purchase_count") \
    .orderBy(desc("purchase_count"))

top_sellers_df.show(5)

# Gold 저장
top_sellers_df.write.mode("overwrite").parquet("s3a://gold/top_selling_items")


# ==========================================
# 분석 3: 퍼널 분석 (Conversion Funnel)
# - 조회(View) -> 장바구니(Cart) -> 구매(Purchase) 전환율 계산
# - 이 코드는 면접에서 "데이터로 비즈니스 문제를 푼 경험"으로 말하기 딱 좋습니다.
# ==========================================
print("3. 퍼널(전환율) 분석 중...")

# Pivot을 이용해 이벤트 타입을 컬럼으로 변환
funnel_df = df \
    .groupBy("device_os") \
    .pivot("event_type", ["view", "cart", "purchase"]) \
    .count() \
    .na.fill(0) # Null 값을 0으로 채움

# 전환율 계산 (구매 / 조회 * 100)
funnel_result_df = funnel_df \
    .withColumn("view_to_cart_rate", (col("cart") / col("view") * 100)) \
    .withColumn("cart_to_purchase_rate", (col("purchase") / col("cart") * 100)) \
    .withColumn("total_conversion_rate", (col("purchase") / col("view") * 100)) \
    .orderBy(desc("total_conversion_rate"))

funnel_result_df.show()

# Gold 저장
funnel_result_df.write.mode("overwrite").parquet("s3a://gold/funnel_analysis")

print("모든 분석 완료! Gold 데이터 적재 끝!")

Silver 데이터 로딩 중...
1. 일별 트래픽 분석 중...
+----------+--------+------------------------+
|      date|total_pv|daily_active_users (DAU)|
+----------+--------+------------------------+
|2026-01-25|    5000|                     101|
+----------+--------+------------------------+

2. 상품 판매 랭킹 분석 중...
+-------+--------------+
|item_id|purchase_count|
+-------+--------------+
|     42|            20|
|     15|            18|
|     47|            17|
|      7|            16|
|     23|            14|
+-------+--------------+
only showing top 5 rows

🔻 3. 퍼널(전환율) 분석 중...
+---------+----+----+--------+------------------+---------------------+---------------------+
|device_os|view|cart|purchase| view_to_cart_rate|cart_to_purchase_rate|total_conversion_rate|
+---------+----+----+--------+------------------+---------------------+---------------------+
|  Android|1167| 340|     171| 29.13453299057412|   50.294117647058826|   14.652956298200515|
|      Web|1201| 346|     166| 28.80932556203164|    47.9768